<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/twelvelabs-io/twelvelabs-developer-experience/blob/main/quickstarts/1.0.0-beta/TwelveLabs_Quickstart_Search.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in  Colab</a>
  </td>
</table>

# Search
This guide shows how to utilize the TwelveLabs Python SDK for searching within your video content.

# Prerequisites
Before you begin, ensure the following prerequisites are met:

- [Sign up](https://playground.twelvelabs.io/) for a free account and obtain your API key from the [API Key](https://playground.twelvelabs.io/dashboard/api-key) page. No credit card is required to use the Free plan. This plan allows you to index up to 600 minutes of videos, which is sufficient for a small project. 
- The videos you wish to upload must meet the requirements in the [Prerequisites](https://docs.twelvelabs.io/v1.3/docs/guides/search#prerequisites) section of the **Search** page.



# Procedure

## Install the TwelveLabs SDK

In [ ]:
%pip install twelvelabs

## Import the required packages

In [9]:
import uuid
from twelvelabs import TwelveLabs
from twelvelabs.indexes import IndexesCreateRequestModelsItem
from twelvelabs.tasks import TasksRetrieveResponse

## Configure your API key


In [ ]:
# For Google Colab, store your API key as a Secret named `TL_API_KEY`. If you don't know how to create a Colab Secret, see https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75.

from google.colab import userdata
TL_API_KEY = userdata.get("TL_API_KEY")

# For other Python environments, you can use environment variables
# TL_API_KEY = os.environ.get('TL_API_KEY')

## Create an index

An index is a basic unit for organizing and storing video data consisting of video embeddings and metadata. Indexes facilitate information retrieval and processing.

When creating a new index, you must specify at least the following information:
- **Name**: Use a brief and descriptive name to facilitate future reference and management.
- **Model configuration**: Provide a list containing the video understanding models and the associated model options you want to enable.

TwelveLabs provides two distinct model types:

- **Embedding models (Marengo)**: This model performs tasks such as search and embedding generation , enabling enhanced video understanding.
- **Generative models (Pegasus)**: This model generates text based on your videos.

Model options determine what information the video understanding model will process:
- **visual**: Analyzes visual content, including actions, objects, events, text (OCR), and brand logos.
- **audio**: Analyzes audio content, including ambient sounds and human speech.

In [ ]:
client = TwelveLabs(api_key=TL_API_KEY)

index = client.indexes.create(
    index_name="<YOUR_INDEX_NAME>",
    models=[
        IndexesCreateRequestModelsItem(
            model_name="marengo2.7",
            model_options=["visual", "audio"]
        )
    ]
)
print(f"Created index: id={index.id}")


## Upload videos to your index

In [ ]:
task = client.tasks.create(
    index_id=index.id, video_url="<YOUR_VIDEO_URL>")
print(f"Created task: id={task.id}")
# 4. Monitor the indexing process
def on_task_update(task: TasksRetrieveResponse):
    print(f"  Status={task.status}")
task = client.tasks.wait_for_done(sleep_interval= 5, task_id=task.id, callback=on_task_update)
if task.status != "ready":
    raise RuntimeError(f"Indexing failed with status {task.status}")
print(
    f"Upload complete. The unique identifier of your video is {task.video_id}.")


## Perform searches

**Using text queries**:

In [ ]:
search_pager = client.search.query(
    index_id=index.id,
    query_text="Steve Jobs",
    search_options=["visual", "audio"],
    # operator="or"
)

print("Search results:")
for clip in search_pager:
    print(
        f" video_id {clip.video_id} score={clip.score} start={clip.start} end={clip.end} confidence={clip.confidence}"
    )

**Using image queries**

In [ ]:
search_pager = client.search.query(
    index_id=index.id,
    query_media_type="image",
    search_options=["visual"],
    query_media_url="<YOUR_IMAGE_URL>"
    # operator="or"
)

print("Search results:")
for clip in search_pager:
    print(
        f" video_id {clip.video_id} score={clip.score} start={clip.start} end={clip.end} confidence={clip.confidence}"
    )